In [1]:
%cd ..

/media/axeh/code_stuff/python_stuff/projects/reaction_diffusion_tutorial


In [2]:
import numpy as np
from scipy.ndimage import convolve
from tqdm import tqdm_notebook as tqdm

from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource
from  ipywidgets import interact
output_notebook()

from recaps.utils import euler_forward, runge_kutta2, convert2img

Loading BokehJS ...

In [3]:
def gierer_meinhardt(t, x, p):
    x = x.reshape(p['size'])   # resize flat 1D array back into (height,width,#channels)-array
    dxdt = np.zeros(p['size']) # preallocate the accumulation terms    
    diffusion = np.zeros(p['size']) # preallocate the diffusion term
    reaction  = np.zeros(p['size']) # preallocate the reaction term
    
    # --- get diffusion term ---
    # this time we'll take into account not only direct horizontal and vertical neighbours,
    # but the diagonal neighbours as well; 
    # they'll be assigned slightely lower weights, since they're a bit far after all
    kernel = np.array([[.05,  .20, .05], 
                       [.20, -1  , .20], 
                       [.05,  .20, .05]]) 
    for k in range(p['size'][2]):   
        diffusion[:,:,k] = p['D'][k]/p['h']**2 * convolve(x[:,:,k], kernel, mode="nearest")
        
    # --- get reaction term ---
    x_a, x_b = x[:,:,0], x[:,:,1]
    reaction[:,:,0] =  p['r']*(x_a**2)/(1 + p['k']*x_a**2)/(x_b+1e-16)  - p['d_a']*x_a + p['r0']
    reaction[:,:,1] =  p['r']* x_a**2 /(1 + p['k']*x_a**2)              - p['d_b']*x_b 
        
    # --- get total accumulation term ---
    dxdt = diffusion + reaction
        
    return dxdt.ravel()

# parameters: system 
p = {'r0' : 0.001,
     'r'  : 0.001,
     'd_a': 0.02,
     'd_b': 0.03,
     'k'  : 0.1,
     'D'  :[0.02,1]}# diffusion rates 

# parameters: spatial grid
resolution = [80,80]          # grid dimensions in pixels
p['size']  = (*resolution, 2) # (height, width, #states)
p['h']     = 0.8              # pixel size in physical units


# time-related
t0, tf, dt = 0, 10000, 0.5
t_span = np.arange(t0, tf+dt, dt)

# initial condition
x_a0 = np.ones(resolution) 
#x_b0 = np.zeros(resolution)
#x_b0[np.random.randint(0,resolution[0],3), np.random.randint(0,resolution[1],3)] = 1 # random seeds (predator)
x_b0 = np.random.rand(*resolution)
x0 = np.stack((x_a0, x_b0), axis=2).ravel()

# run the simulation: use euler_forward or runge_kutta2 
img_development = runge_kutta2(gierer_meinhardt, x0, t_span, p)

  0%|          | 0/20000 [00:00<?, ?it/s]

In [4]:
# plot stuff!
img = np.zeros(p['size'][:2], dtype=np.uint32)
view = img.view(dtype=np.uint8).reshape((*p['size'][:2], 4))
view += np.flipud(convert2img(img_development[:,0], p['size']))

pb = figure(
    x_range=(0,p['size'][0]), 
    y_range=(0,p['size'][1]),
    plot_width=p['size'][0]*4,
    plot_height=p['size'][1]*4
)
r = pb.image_rgba(
    image=[img],
    x=0, 
    y=0, 
    dw=p['size'][0], 
    dh=p['size'][1]
)
handle = show(pb, notebook_handle=True)  

def update(t=0):
    i = int(t/dt)
    # update data_source
    #(adjust the contrst to make patterns more visible)
    img = np.zeros(p['size'][:2], dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((*p['size'][:2], 4))
    view += np.flipud(convert2img(img_development[:,i], p['size'], steepness=4, midpoint=0.8))
    r.data_source.data['image'] = [img]
    push_notebook(handle=handle)
    
interact(update, t=(t0,tf,dt));

interactive(children=(FloatSlider(value=0.0, description='t', max=10000.0, step=0.5), Output()), _dom_classes=…